# <center>3.4.1 - Regularization</center>

<center> <img src = '../images/lasso_ridge.png'> </center>

Лассо- и ридж-регрессии. [Источник](https://hastie.su.domains/Papers/ESLII.pdf)

Для победы в соревнованиях по машинному обучению вы редко будете использовать линейную регрессию, но на этой простой модели легко показать принципы регуляризации и отбора признаков, которые в той или иной форме встречаются в других более сложных алгоритмах. Хорошо разобравшись в этой теме, уже можно обучать устойчивые модели, которые не будут улетать вниз на приватной части лидерборда, а также эти знания пригодятся и в продакшен моделях, которые не будут ломаться от каждого изменения в данных.<br>
Постараемся в этом ноутбуке обойтись без формул и доказательств, а сразу посмотреть всё на примерах.

У модели два основных источника ошибок:
* **Дисперсия** (variance) - ошибка, связанная с чувствительностью к малейшим изменениям в обучающих данных.
* **Смещение** (bias) - ошибка, связанная с неверными предположениями модели относительно связи признаков и таргета.

Если сумма этих ошибок будет минимальной, то получим на выходе максимально эффективную устойчивую модель.

<center> <img src = '../images/overfitting-vs-underfitting.png'> </center>

Рассмотрим формулу линейной регрессии:
* $Y_i$ - зависимая переменная (таргет)
* $X_i$ - независимая переменная (признак, предиктор)
* $\beta$ - коэффициенты, при домножении $X$ на которые получаем искомый прогноз, причем $\beta_0$ - свободный член (константа, intercept), равен 0, если функция проходит через начало координат. 

<center> <img src = https://www.isixsigma.com/wp-content/uploads/2018/11/linear_regression_transfer_function.png height=400 width=500> </center>

### Обычно задача регрессии формулируется так:
Имея набор признаков (обычно матрица $X$), требуется найти набор коэффициентов (обычно вектор $\beta$), которые нужно умножить на значения $Х$, чтобы получить предсказание (обычно вектор $Y$).

<center> <img src = '../images/formule.png' height=500 width=700> </center>

Довольно часто при обучении регрессии происходит "черезмерно близкая подгонка" данных (оверфит). В этом случае нам помогут методы регуляризации.

## Ридж- и Лассо-регрессии
`Ридж и Лассо (Ridge & Lasso)`   —  это модели линейной регрессии, но с поправочными (штрафными) коэффициентами, также называемыми регуляризацией. Они вносят поправки в размерность $\beta$-вектора разными способами.
* **Лассо-регрессия (L1-регуляризация)** — накладывает штраф на L1-норму $\beta$-вектора. L1-норма вектора  —  это сумма абсолютных значений в этом векторе. Модель пытается достичь большей точности, путем нахождения и отбрасывания бесполезных коэффициентов.
* **Ридж-регрессия (L2-регуляризация)** — накладывает штраф на L2-норму $\beta$-вектора. L2-норма вектора  —  это квадратный корень из суммы квадратов значений в векторе. Ридж-регрессия не позволяет коэффициентам $\beta$-вектора достигать экстремальных значений (что часто происходит при переобучении). Модель пытается достичь большей точности, при этом ни один коэффициент не должен достигать экстремальных значений.

Оба эти метода имеют **коэффициент регуляризации** (называемый “лямбда”), который контролирует величину штрафа. При $\lambda=0$ как лассо-, так и ридж-регрессия становятся моделями линейной регрессии (в этом случае просто не накладываются никакие штрафы). При увеличении лямбды возрастает ограничение на размер $\beta$-вектора. При этом каждая регрессия оптимизирует его по-своему, пытаясь подобрать наилучший набор коэффициентов с учетом собственных ограничений.<br>
Переходим от теории к практике.

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Загружаем train-датасет который мы сохранили на шаге quickstart
rides_info = pd.read_csv('../data/quickstart_train.csv')
rides_info.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,32.22,engine_fuel,3.880920,1.651883e+07,0.1,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,27.51,engine_fuel,4.181149,1.398317e+07,0.1,203.462289,-14.260456,174,171


In [3]:
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

In [4]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

Также важно отметить, что признаки следует отнормировать перед подачей в линейную модель. Также без нормировки не будет работать регуляризация.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
target_scaler = StandardScaler()
df = scaler.fit_transform(rides_info.drop(drop_cols, axis=1))
target = target_scaler.fit_transform(rides_info['target_reg'].values.reshape(-1, 1))

In [5]:
X = rides_info.drop(drop_cols, axis=1)
y = rides_info['target_reg']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear Regression

In [7]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
y_pred = linear_reg.predict(X_test)

# посчитаем RMSE, чтобы избавиться от ошибки в квадрате (что это за ошибка дни в квадрате)
rmse_linear = mean_squared_error(y_pred, y_test) ** 0.5

print(rmse_linear)

14.562509138703911


In [8]:
# Посмотрим на вектор весов
print(linear_reg.coef_)

[ 2.88087629e-01 -1.47064987e-01  7.37558446e-06  1.75613018e-01
  1.26243062e+01 -3.55794645e-07  1.08577758e+01  3.24482039e-01
  1.08684847e-01  2.41584530e-13 -1.20121796e+00  2.97857360e+00
 -1.82927630e+00  9.23232090e-02 -1.24162051e+00  4.61756336e-01
 -4.61756336e-01  4.23255447e+00  8.42750125e+00  8.63650367e-01
 -3.49578759e-01 -2.82502348e+00 -1.11253041e+00  2.05235742e+00
 -2.14072616e-01 -8.31091154e-01 -1.75816704e-01  1.38931392e+01
 -5.50415913e+00 -4.53951548e+00  2.30371124e+00  7.36789945e-01
 -2.08662389e+00  2.84030138e-01 -3.61732658e-01  1.32633767e+00
  1.12230379e-01 -1.69230884e+00 -2.93892641e+00 -1.73186984e+00
 -5.63547818e-01  1.18695156e+00 -1.04924565e+01]


In [9]:
# Посмотрим на свободный член бэта0
print(linear_reg.intercept_)

87.57368293696851


### Lasso Regression

In [10]:
lambda_values = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05,  0.1, 0.2, 0.3, 0.4, 0.5]

for lambda_val in lambda_values:
    lasso_reg = Lasso(lambda_val)
    lasso_reg.fit(X_train, y_train)
    y_pred = lasso_reg.predict(X_test)
    #y_pred = target_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
    #y_true = target_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))
    rmse_lasso = mean_squared_error(y_pred, y_test) ** 0.5
    #mse_lasso = mean_squared_error(y_pred, y_test)
    print(("Lasso MSE with Lambda={} is {}").format(lambda_val, rmse_lasso))

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.966e+04, tolerance: 5.723e+01
  model = cd_fast.enet_coordinate_descent(


Lasso MSE with Lambda=1e-05 is 14.562531337396315
Lasso MSE with Lambda=0.0001 is 14.562865207580808
Lasso MSE with Lambda=0.001 is 14.566221980543073
Lasso MSE with Lambda=0.005 is 14.578707717809753
Lasso MSE with Lambda=0.01 is 14.587063886816646
Lasso MSE with Lambda=0.05 is 14.632224481072923
Lasso MSE with Lambda=0.1 is 14.678577433561504
Lasso MSE with Lambda=0.2 is 14.831395323440448
Lasso MSE with Lambda=0.3 is 14.937077750584296
Lasso MSE with Lambda=0.4 is 14.977851978983788
Lasso MSE with Lambda=0.5 is 15.005633595052027


In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   car_rating                2337 non-null   float64
 1   year_to_start             2337 non-null   int64  
 2   riders                    2337 non-null   int64  
 3   year_to_work              2337 non-null   int64  
 4   mean_rating               2337 non-null   float64
 5   distance_sum              2337 non-null   float64
 6   rating_min                2337 non-null   float64
 7   speed_max                 2337 non-null   float64
 8   user_ride_quality_median  2337 non-null   float64
 9   deviation_normal_count    2337 non-null   int64  
 10  user_uniq                 2337 non-null   int64  
 11  car_type_business         2337 non-null   uint8  
 12  car_type_economy          2337 non-null   uint8  
 13  car_type_premium          2337 non-null   uint8  
 14  car_type

In [11]:
# Посмотрим на вектор весов при самом большом значении лямбда
print(lasso_reg.coef_)

[ 0.00000000e+00 -0.00000000e+00  2.23656591e-06  1.48373258e-01
  9.82163697e+00 -4.04185024e-07  0.00000000e+00  2.91162882e-01
  9.35758139e-02  0.00000000e+00 -1.08874728e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00]


Чем больше лямбда, тем больше фичей зануляется, при этом точность не сильно упала (всего на 0.5).
Как видно, модель обнулила 80% коэффициентов, оставив 8 из них. Для одного из признаков (`mean_rating`) сохранился достаточно большой вес. <br>
Другими словами, мы “приказали” лассо-регрессии найти наилучшую модель, учитывая ограничения на то, какой вес можно придать каждому коэффициенту (т.е. “бюджет”). Поэтому она “решила” приложить большую часть этого “бюджета” к среднему рейтингу, чтобы определить когда машина сломается.

In [17]:
# Посмотрим на вектор весов при самом низком значении лямбда
lasso_reg = Lasso(lambda_values[0])
lasso_reg.fit(X_train, y_train)

print(lasso_reg.coef_)

[ 2.88104946e-01 -1.46906355e-01  7.36890966e-06  1.75609645e-01
  1.26242892e+01 -3.55799201e-07  1.08572978e+01  3.24481907e-01
  1.08682523e-01  0.00000000e+00 -1.20102589e+00  5.29166001e+00
 -8.47978481e-01  2.56663861e+00 -1.89698308e-01  2.82490775e+00
 -1.19174823e-12  3.19230589e+00  7.38750850e+00 -1.74119498e-01
 -1.22517642e+00 -3.70156937e+00 -6.58186788e-01  2.50627453e+00
  2.39947245e-01 -3.76739807e-01  2.07606587e-01  1.11132262e+01
 -6.38055826e+00 -5.41495441e+00  1.42494854e+00  1.12018524e+00
 -1.70292494e+00  6.67433043e-01  9.22293725e-02  1.78021231e+00
  5.66196655e-01 -1.23807417e+00 -5.87173694e+00 -1.27755087e+00
 -1.09317457e-01  1.64099625e+00 -1.34265637e+01]


При низком значении лямбды занулился только 1 признак, но модель показала самую высокую точность.

### Ridge Regression

Следует помнить, что значения лямбды у ридж- и лассо-регрессий не пропорциональны, т.е. лямбда 5 для лассо-регрессии ни в коем случае не равна лямбде 5 для ридж-регрессии.


In [18]:
lambda_values = [0.00001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 5, 6, 7, 8, 9, 10]

for lambda_val in lambda_values:
    ridge_reg = Ridge(lambda_val)
    ridge_reg.fit(X_train, y_train)
    y_pred = ridge_reg.predict(X_test)
    rmse_ridge = mean_squared_error(y_pred, y_test) ** 0.5
    print(("Lasso MSE with Lambda={} is {}").format(lambda_val, rmse_ridge))

Lasso MSE with Lambda=1e-05 is 14.562509249645005
Lasso MSE with Lambda=0.01 is 14.56262003624121
Lasso MSE with Lambda=0.05 is 14.563062643911982
Lasso MSE with Lambda=0.1 is 14.56361366436038
Lasso MSE with Lambda=0.5 is 14.567927794980239
Lasso MSE with Lambda=1 is 14.573072267667612
Lasso MSE with Lambda=1.5 is 14.577934023113965
Lasso MSE with Lambda=3 is 14.590908837556091
Lasso MSE with Lambda=5 is 14.605048920001117
Lasso MSE with Lambda=6 is 14.611057201766233
Lasso MSE with Lambda=7 is 14.616503299811134
Lasso MSE with Lambda=8 is 14.62147487456486
Lasso MSE with Lambda=9 is 14.626044668305562
Lasso MSE with Lambda=10 is 14.6302729405031


In [19]:
# Вектор весов при высокой лямбда
print(ridge_reg.coef_)

[ 3.32096994e-01 -1.50849264e-01  8.43435004e-06  1.79872879e-01
  1.23345909e+01 -3.68037181e-07  8.97384865e+00  3.20562428e-01
  1.04929922e-01  0.00000000e+00 -9.85576156e-01  2.41447288e+00
 -2.02200941e+00  9.94439846e-01 -1.38690331e+00  2.09105737e+00
 -2.09105737e+00  1.76873902e+00  4.54397883e+00 -1.21869890e-01
  1.92525834e-01 -1.54749288e+00 -9.92209838e-01  1.79756277e+00
 -1.74624366e-01 -7.62421482e-01 -1.49420630e-01  7.28746548e+00
 -3.20457811e+00 -1.92395981e+00  1.61051237e+00  5.52109592e-01
 -1.99204006e+00  2.02447784e-01 -3.47748469e-01  1.09084058e+00
  9.00700576e-02 -1.65732619e+00 -5.58763336e-01 -1.59412231e+00
 -6.14207480e-01  1.14217732e+00 -4.63764478e+00]


In [20]:
# Вектор весов при низкой лямбда
ridge_reg = Lasso(lambda_values[0])
ridge_reg.fit(X_train, y_train)

print(ridge_reg.coef_)

[ 2.88104946e-01 -1.46906355e-01  7.36890966e-06  1.75609645e-01
  1.26242892e+01 -3.55799201e-07  1.08572978e+01  3.24481907e-01
  1.08682523e-01  0.00000000e+00 -1.20102589e+00  5.29166001e+00
 -8.47978481e-01  2.56663861e+00 -1.89698308e-01  2.82490775e+00
 -1.19174823e-12  3.19230589e+00  7.38750850e+00 -1.74119498e-01
 -1.22517642e+00 -3.70156937e+00 -6.58186788e-01  2.50627453e+00
  2.39947245e-01 -3.76739807e-01  2.07606587e-01  1.11132262e+01
 -6.38055826e+00 -5.41495441e+00  1.42494854e+00  1.12018524e+00
 -1.70292494e+00  6.67433043e-01  9.22293725e-02  1.78021231e+00
  5.66196655e-01 -1.23807417e+00 -5.87173694e+00 -1.27755087e+00
 -1.09317457e-01  1.64099625e+00 -1.34265637e+01]


Как видите, коэффициент для `mean_rating` уже не такой высокий, в то время как все остальные коэффициенты уменьшились. Однако, только один из них был обнулен, в отличие от лассо-регрессии. Также заметно, что точность модели почти не меняется при изменении лямбда (скорее всего из-за специфики данных).

В этом заключается ключевое различие между двумя методами: лассо-регрессия часто обнуляет признаки, а ридж-регрессия уменьшает вес большинства из них в модели.

Поэтому стоит просматривать бета-векторы каждой модели и перепроверять значения: понимание того, что происходит в бета-векторе, является ключом к пониманию этих моделей.

### Elastic-Net - COMBO L1 + L2
Посмотрим описание из документации библиотеки [sklearn](https://scikit-learn.ru/1-1-linear-models/#elastic-net):
>ElasticNet is a linear regression model trained with both and -norm regularization of the coefficients. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of and using the l1_ratio parameter.
Elastic-net is useful when there are multiple features that are correlated with one another. Lasso is likely to pick one of these at random, while elastic-net is likely to pick both. A practical advantage of trading-off between Lasso and Ridge is that it allows Elastic-Net to inherit some of Ridge’s stability under rotation.

>`ElasticNet` полезна, когда есть несколько признаков, которые коррелируют друг с другом. Лассо, вероятно, выберет один из них наугад, а elastic-net — и то, и другое.

In [21]:
l1_ratios = [0.00001, 0.001, 0.01, 0.05, 0.1, 0.5, 0.7, 0.9, 1]

for l1_ratio in l1_ratios:
    elasticnet_reg = ElasticNet(l1_ratio=l1_ratio)
    elasticnet_reg.fit(X_train, y_train)
    y_pred = elasticnet_reg.predict(X_test)
    rmse_elastic = mean_squared_error(y_pred, y_test) ** 0.5
    print(("ElasticNet MSE with l1_ratio={} is {}").format(l1_ratio, rmse_elastic))

ElasticNet MSE with l1_ratio=1e-05 is 15.521003805723765
ElasticNet MSE with l1_ratio=0.001 is 15.52103383273258
ElasticNet MSE with l1_ratio=0.01 is 15.52154412248223
ElasticNet MSE with l1_ratio=0.05 is 15.524490800044598
ElasticNet MSE with l1_ratio=0.1 is 15.527894342826968
ElasticNet MSE with l1_ratio=0.5 is 15.524286170666144
ElasticNet MSE with l1_ratio=0.7 is 15.485878593163134
ElasticNet MSE with l1_ratio=0.9 is 15.384412021567083
ElasticNet MSE with l1_ratio=1 is 15.22755055292332


In [17]:
# При высоком значении l1_ratio
print(elasticnet_reg.coef_)

[ 0.          0.          0.          0.          3.32352232 -0.60286002
  0.80273328  8.33769347  0.          0.         -2.32946425  0.
 -0.          0.29651072 -0.          0.23765759 -0.          0.
  0.          0.          0.          0.         -0.          0.
 -0.         -0.          0.          0.84630523 -0.          0.
  0.          0.         -0.          0.         -0.          0.
 -0.         -0.          0.         -0.         -0.          0.
 -0.        ]


`ElasticNet` присвоил большие коэффициенты 2 признакам `mean_rating` и `speed_max` и занулил большинство фичей.

In [22]:
# При низком значении l1_ratio
elasticnet_reg = ElasticNet(l1_ratio=l1_ratios[0])
elasticnet_reg.fit(X_train, y_train)
print(elasticnet_reg.coef_)

[ 2.20143687e-01  5.61252346e-03  1.68286708e-07  2.19162534e-01
  1.47365163e+00 -4.61771802e-07  4.11666318e-01  2.32428307e-01
  8.52749684e-02  0.00000000e+00 -9.89820806e-01  2.64921465e-01
 -2.76617416e-01  1.42562753e-01 -1.30866802e-01  1.98958188e-01
 -1.98958188e-01  3.95489150e-02  8.96639025e-02  1.03539870e-02
  3.05439315e-02  6.52737650e-03 -5.78935917e-02  7.47719748e-02
 -2.57143753e-02 -7.69603299e-02 -8.79465243e-04  1.95982239e-01
 -1.57406643e-02 -1.42660242e-04  4.77412421e-02 -1.98423478e-02
 -1.24775718e-01  1.46407289e-02  5.88515990e-03  2.48453365e-02
  1.60444912e-03 -1.37352026e-01  2.22436185e-02 -1.01382663e-01
 -4.17673519e-02  5.73460018e-02 -1.92676700e-02]


## Что же выбрать?
* **Лассо (L1-регуляризацию)** следует использовать, когда есть несколько характеристик с высокой предсказательной способностью, а остальные бесполезны. Она обнуляет бесполезные характеристики и оставляет только подмножество переменных.

* **Ридж (L2-регуляризацию)** лучше применять, когда предсказательная способность набора данных распределена между различными характеристиками. Ридж-регрессия не обнуляет характеристики, которые могут быть полезны при составлении прогнозов, а просто уменьшает вес большинства переменных в модели.

На практике это обычно трудно определить поэтому лучше экспериментировать на тестовом множестве, используя различные значения лямбды.
### Зачем же в итоге нужна регуляризация?
* Регуляризация предназначена для регулирования сложности модели и её целью является упрощение модели.
* Регуляризация помогает бороться с переобучением и увеличивает обобщающую способность (робастность) модели.
* Регуляризация применяется, когда независимые переменные (признаки) коррелируют друг с другом, т.е. имеет место быть мультиколлинеарность признаков.
* Регуляризация работает даже при полной мультиколлинеарности признаков.

Как уже отмечалось в начале, вы вряд ли будете использовать линейную регрессию и её модификации с регуляризацией в соревнованиях, но параметры, отвечающие за регуляризацию, присутствуют во многих популярных библиотеках и моделях машинного обучения (`CatBoost`, `LightGBM`, `RandomForest`, `SVM`, и многие другие). Надеемся, что освоив этот урок, вы будете более осознанно подходить к тюнингу того или иного параметра, а не просто рандомно перебирать цифры, пытаясь угадать заветную комбинацию 😁.

## Полезные ссылки:
1. Отличный [курс от ODS](https://ods.ai/tracks/linear-models-spring22) про линейные модели и регуляризацию со всеми математическими выкладками.
2. Если хотите углубиться в математику (это поможет понять, как работает регуляризация), прочитайте главу 3.4 в книге [“Элементы статистического обучения”](https://hastie.su.domains/Papers/ESLII.pdf), написанной Треворой Хасти, Робертом Тибширани и Джеромом Фридманом. Роберт Тибширани  —  автор метода лассо-регрессии.
3. [Англоязычная статья](https://towardsdatascience.com/lasso-and-ridge-regression-an-intuitive-comparison-3ee415487d18), которая легла в основу этого ноутбука.